In [5]:
from sklearn.tree import DecisionTreeClassifier
from util import corpus_to_df,read_corpus
import numpy as np
from joblib import Memory
from shutil import rmtree
from sklearn.model_selection import train_test_split
from os import path
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from pydotplus.graphviz import graph_from_dot_data
from sklearn.tree import export_graphviz
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
def document(corpus):
    return (corpus.raw(id) for id in corpus.fileids())

In [3]:
corpus_en_cleaned,corpus_nl_cleaned,corpus_cleaned = read_corpus()
len(corpus_en_cleaned.fileids()), len(corpus_nl_cleaned.fileids())

(900, 3884)

In [4]:
df = corpus_to_df(corpus_cleaned)
df.head()

,id,lang,innov,innov_
0,en/innov/10000232.txt,en,innov,1
1,en/innov/10000364.txt,en,innov,1
2,en/innov/10010653.txt,en,innov,1
3,en/innov/10013385.txt,en,innov,1
4,en/innov/10025316.txt,en,innov,1


In [85]:
y = df['innov_']

In [87]:
c_vec = CountVectorizer(min_df = 2, max_df = 0.95,binary=True)

In [89]:
x = c_vec.fit_transform(document(corpus_cleaned))

In [91]:
select = SelectKBest(chi2,k=2000)

In [92]:
x = select.fit_transform(x,y)

In [93]:
DecisionTreeClassifier?

In [94]:
clf = DecisionTreeClassifier(criterion='entropy')

In [97]:
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify = y, test_size = 0.1)

In [98]:
x_train.toarray().shape, y_train.shape

((4337, 2000), (4337,))

In [99]:
clf.fit(x_train.toarray(), y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [101]:
select.get_support()

array([False, False, False, ..., False, False, False])

In [102]:
feature_names = np.array(c_vec.get_feature_names())[select.get_support()]

In [106]:
# dot_data = export_graphviz(                           # Create dot data
#     clf, filled=True, rounded=True,
#     class_names=['other', 'innov'],
#     feature_names= feature_names,
#     out_file=None
# )

# graph = graph_from_dot_data(dot_data)                 # Create graph from dot data
# graph.write_png('tree.png')  

In [108]:
print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.48      0.50      0.49       219
           1       0.57      0.56      0.56       263

    accuracy                           0.53       482
   macro avg       0.53      0.53      0.53       482
weighted avg       0.53      0.53      0.53       482



In [112]:
ens = BaggingClassifier(clf, n_estimators=100,random_state = 42, bootstrap_features=True)

In [113]:
ens.fit(x_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=Non

In [114]:
print(classification_report(y_test, ens.predict(x_test)))

              precision    recall  f1-score   support

           0       0.60      0.47      0.53       219
           1       0.63      0.75      0.68       263

    accuracy                           0.62       482
   macro avg       0.61      0.61      0.60       482
weighted avg       0.62      0.62      0.61       482



In [115]:
from sklearn.pipeline import Pipeline

In [137]:
clf_p = Pipeline([
    ('vec', CountVectorizer(min_df = 2, max_df = 0.95,binary=True)),
    ('clf', BaggingClassifier(DecisionTreeClassifier(criterion='entropy'), n_estimators=100,max_samples=0.5,max_features=0.1, n_jobs=-1, random_state = 42))    
])

In [138]:
df_train, df_test = train_test_split(df, stratify = df['innov'], test_size = 0.1)

In [139]:
def read_document(ids, corpus):
    return [corpus.raw(id) for id in ids]

In [140]:
clf_p.fit(read_document(df_train['id'],corpus_cleaned), df_train['innov_'])

Pipeline(memory=None,
         steps=[('vec',
                 CountVectorizer(analyzer='word', binary=True,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.95,
                                 max_features=None, min_df=2,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None...
                                                                         min_impurity_decrease=0.0,
                                                                         min_impurity_split=None,
                                                                         min_samples_leaf=1,
                                             

In [144]:
from sklearn.model_selection import cross_val_score

In [141]:
print(classification_report(df_test['innov_'],clf_p.predict(read_document(df_test['id'],corpus_cleaned)),target_names=['other','innov']))

              precision    recall  f1-score   support

       other       0.61      0.54      0.58       219
       innov       0.65      0.71      0.68       263

    accuracy                           0.64       482
   macro avg       0.63      0.63      0.63       482
weighted avg       0.63      0.64      0.63       482



In [145]:
scores = cross_val_score(clf_p, read_document(df_test['id'],corpus_cleaned), df_test['innov_'])

In [147]:
np.mean(scores), np.std(scores)

(0.5933848797250859, 0.03204382421048147)

In [148]:
import pickle

In [150]:
with open('ensemble.pickle', 'wb') as f:
    pickle.dump(clf_p, f)

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
clf_p2 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 2, max_df = 0.95)),
    ('clf', BaggingClassifier(DecisionTreeClassifier(criterion='entropy'), n_estimators=100,max_samples=0.5,max_features=0.1, n_jobs=-1, random_state = 42))    
])

In [153]:
clf_p2.fit(read_document(df_train['id'],corpus_cleaned), df_train['innov_'])

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.95, max_features=None,
                                 min_df=2, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern=...
                                                                         min_impurity_decrease=0.0,
                                                                         min_impurity_split=None,
                                                                         min_samples_leaf=1,
           

In [154]:
print(classification_report(df_test['innov_'],clf_p2.predict(read_document(df_test['id'],corpus_cleaned)),target_names=['other','innov']))

              precision    recall  f1-score   support

       other       0.62      0.54      0.58       219
       innov       0.65      0.72      0.69       263

    accuracy                           0.64       482
   macro avg       0.64      0.63      0.63       482
weighted avg       0.64      0.64      0.64       482



In [162]:
_ = clf_p[1]

In [168]:
names = _.estimators_features_[0]

In [173]:
x_transform = clf_p[0].transform(read_document(df_train['id'], corpus_cleaned))

In [187]:
_ = clf_p[1]

In [190]:
np.array(_.estimators_features_).shape

(100, 13871)

In [194]:
all_names = []
for f in clf_p[1].estimators_features_:
    names = f[:5]
    all_names.extend(np.array(clf_p[0].get_feature_names())[names])        

In [195]:
all_names = np.array(all_names)

In [196]:
all_names.shape

(500,)

In [ ]:
pipe2.fit()

In [33]:
voting = VotingClassifier(estimators=[
    ('lr', LogisticRegression(random_state=42)),
    ('rf', clf_p2[1]),
    ('mnb', MultinomialNB())
], voting = 'soft')

In [34]:
clf_p3 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 2, max_df = 0.95)),
    ('clf', voting)])

In [6]:
import pandas as pd

# start here

In [29]:
corpus_cleaned = read_corpus('/Users/septem/Downloads/companies_cleaned/')[2]
df = pd.read_csv('./companies.csv')
from scipy import stats
df = df[np.abs(stats.zscore(df['text_length'])) < 3]
df['text'] = [corpus_cleaned.raw(id) for id in df['id']]
df = df[df['lang'] != 'rest']
onehot = OneHotEncoder() 
scaler = MinMaxScaler()
lang_feature = onehot.fit_transform([[lang] for lang in df['lang']])
stat_feature = scaler.fit_transform(np.array(df[['avg_word_len']]))
df_small = np.concatenate((lang_feature.toarray(), stat_feature,df['text'][:,np.newaxis]), axis = 1)
df_small = pd.DataFrame(df_small)
df_small = df_small.set_index(df.index)
df_small['innov_'] = df['innov_']
x_train, x_test, y_train, y_test = train_test_split(df.index, df['innov_'], random_state = 42, stratify=df['innov_'], test_size = 0.1)
x_train = df_small.loc[x_train]
x_test = df_small.loc[x_test]
x_train = x_train.drop(columns='innov_')
x_test = x_test.drop(columns='innov_')
df.head()

,BEID,id,lang,innov,innov_,word_count,text_length,avg_word_len,URL,text
0,10000232,en/innov/10000232.txt,en,innov,1,8520,49903,5.857160,http://www.vdlbuscoach.com/,expertis travel travel ever experi motto make ...
1,10000364,en/innov/10000364.txt,en,innov,1,11801,68869,5.835861,https://www.stork.com/en/products-services/pow...,gear servic stork gear servic stork gear servi...
2,10010653,en/innov/10010653.txt,en,innov,1,373,1877,5.032172,http://www.oriental-city.com/,boekingsbureau orient citi zeven dagen per wee...
3,10013385,en/innov/10013385.txt,en,innov,1,2889,18845,6.523018,https://www.honeywell-buildingsolutions.nl/,cybersecur honeywel comfortpoint open ​airport...
4,10025316,en/innov/10025316.txt,en,innov,1,102,564,5.529412,http://www.winteb.nl/contact/,product servic win hia wiko type wiko goosenec...


In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.svm import LinearSVC
pipe = Pipeline([
    ('union', ColumnTransformer([
        ('vect', Pipeline([
            ('tfidf',TfidfVectorizer()),
            ('select', SelectKBest(f_classif, k = 20000))
        ]), 3),
    ], remainder='passthrough')),
    
    ('clf', VotingClassifier(estimators=[
    ('lr', LogisticRegression(C=1,max_iter=1000, solver='liblinear', penalty='l2', random_state=42)),
    ('svm', LinearSVC()),
    ('rf', clf_p2[1]),
    ('mnb', MultinomialNB())
], voting = 'soft'))
])
pipe.fit(x_train, y_train)
print(classification_report(y_test, pipe.predict(x_test)))


              precision    recall  f1-score   support

           0       0.64      0.28      0.39       216
           1       0.59      0.87      0.70       261

    accuracy                           0.60       477
   macro avg       0.61      0.57      0.55       477
weighted avg       0.61      0.60      0.56       477



In [37]:
clf_p3.fit(x_train[3], y_train)
print(classification_report(y_test, clf_p3.predict(x_test[3])))

              precision    recall  f1-score   support

           0       0.65      0.37      0.47       216
           1       0.62      0.84      0.71       261

    accuracy                           0.62       477
   macro avg       0.63      0.60      0.59       477
weighted avg       0.63      0.62      0.60       477



In [208]:
clf_p3.fit(read_document(df_train['id'],corpus_cleaned), df_train['innov_'])

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.95, max_features=None,
                                 min_df=2, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern=...
                                                                                                       presort='deprecated',
                                                                                                       random_state=None,
                                                       

In [209]:
print(classification_report(df_test['innov_'],clf_p3.predict(read_document(df_test['id'],corpus_cleaned)),target_names=['other','innov']))

              precision    recall  f1-score   support

       other       0.75      0.35      0.47       219
       innov       0.62      0.90      0.74       263

    accuracy                           0.65       482
   macro avg       0.69      0.63      0.61       482
weighted avg       0.68      0.65      0.62       482



In [ ]:
from sklearn.feature_selection import f_classif

In [219]:
p4 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('select', SelectKBest(f_classif, 20000)),
    ('lg', LogisticRegression())
])

In [ ]:
pipe2 = Pipeline([
    ('union', ColumnTransformer([
        ('vect', Pipeline([
            ('tfidf',TfidfVectorizer(min_df=2,max_df=0.8)),
            ('select', SelectKBest(f_classif, k = 20000))
        ]), 3),
    ], remainder='passthrough')),
    ('clf', LogisticRegression(C=1,max_iter=1000, solver='liblinear', penalty='l2', random_state=42))
])

In [216]:
x_train, x_test, y_train, y_test = split_data(corpus_cleaned)

In [217]:
p4.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                 vocabulary=None)),
                ('select',
                 SelectKBest(k=20000,
                             score_func=<function f_classif at 0x1a3056c950>)),
                ('lg',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
        

In [230]:
kwargs = {
        'ngram_range' : (1,1),
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : 'word',
        'min_df' : 2,
        'max_df' : 0.95
    }


In [231]:
p4 = Pipeline([
    ('tfidf', TfidfVectorizer(**kwargs)),
    ('select', SelectKBest(f_classif, 20000)),
    ('lg', LogisticRegression(solver='liblinear', C=1, penalty='l2',max_iter=1000, random_state=42))
])
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p4.fit(x_train, y_train)
print(classification_report(y_test, p4.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.61      0.47      0.53       218
           1       0.63      0.75      0.69       264

    accuracy                           0.63       482
   macro avg       0.62      0.61      0.61       482
weighted avg       0.62      0.63      0.62       482



In [232]:
p4 = Pipeline([
    ('tfidf', TfidfVectorizer(**kwargs)),
    ('lg', LogisticRegression(solver='liblinear', C=1, penalty='l2',max_iter=1000,random_state=42))
])
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p4.fit(x_train, y_train)
print(classification_report(y_test, p4.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.63      0.49      0.55       218
           1       0.64      0.76      0.70       264

    accuracy                           0.64       482
   macro avg       0.64      0.63      0.62       482
weighted avg       0.64      0.64      0.63       482



In [236]:
p4 = Pipeline([
    ('tfidf', TfidfVectorizer(**kwargs)),
    ('lg', LogisticRegression(solver='liblinear', C=1, penalty='l2',max_iter=1000,random_state=42))
])
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p4.fit(x_train, y_train)
print(classification_report(y_test, p4.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.64      0.46      0.53       219
           1       0.63      0.79      0.70       263

    accuracy                           0.64       482
   macro avg       0.64      0.62      0.62       482
weighted avg       0.64      0.64      0.63       482



In [276]:
tuning_params = dict(
    tfidf__max_df = [0.85,0.9,0.95],
#     tfidf__min_df = [2,5,10],
#     ngram_range = [(1,1),(1,2)],
    lg__C = np.logspace(-1,4,5),
    lg__penalty = ['l1', 'l2'],
#     lg__max_iter = [20,40,100]
)

In [277]:
tuning_params

{'tfidf__max_df': [0.85, 0.9, 0.95],
 'lg__C': array([1.00000000e-01, 1.77827941e+00, 3.16227766e+01, 5.62341325e+02,
        1.00000000e+04]),
 'lg__penalty': ['l1', 'l2']}

In [278]:
grid = GridSearchCV(p4, param_grid = tuning_params, n_jobs = -1,verbose=True)

In [279]:
from joblib import Memory
from shutil import rmtree
location = 'cachedir'
memory = Memory(location=location, verbose=10)
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
grid.fit(list(x_train), y_train)
memory.clear(warn=False)
rmtree(location)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 44.6min finished
/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [280]:
grid.best_params_

{'lg__C': 1.7782794100389228, 'lg__penalty': 'l2', 'tfidf__max_df': 0.85}

In [293]:
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
print(classification_report(y_test,grid.best_estimator_.predict(x_test)))

              precision    recall  f1-score   support

           0       0.63      0.48      0.54       219
           1       0.64      0.76      0.70       263

    accuracy                           0.63       482
   macro avg       0.63      0.62      0.62       482
weighted avg       0.63      0.63      0.63       482



In [302]:
def make_data(corpus):
    df = corpus_to_df(corpus)
    x = [corpus.raw(id) for id in df['id']]
    y = df['innov_']
    return x,y


In [310]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='replace', dtype='int32',
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.85, max_features=None,
                                 min_df=2, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='unicode',
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lg',
                 LogisticRegression(C=1.7782794100389228, class_weight=None,
                                    dual=False, fit_intercept=True,
                                    intercept

In [303]:
x,y = make_data(corpus_cleaned)
scores = cross_val_score(grid.best_estimator_, x,y,cv=5)

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [304]:
np.mean(scores), np.std(scores)

(0.6109129061585726, 0.0148606506934016)

In [307]:
p5 = Pipeline([
    ('tfidf', TfidfVectorizer(**kwargs)),
    ('lg', LogisticRegression(solver='liblinear', C=1, penalty='l2',max_iter=1000,random_state=42))
])

In [321]:
p5.set_params(**grid.best_params_)
p5.set_params(lg__C=1)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='replace', dtype='int32',
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.85, max_features=None,
                                 min_df=2, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='unicode',
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lg',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ra

In [322]:
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p5.fit(x_train, y_train)
print(classification_report(y_test, p5.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.64      0.46      0.53       219
           1       0.63      0.79      0.70       263

    accuracy                           0.64       482
   macro avg       0.64      0.62      0.62       482
weighted avg       0.64      0.64      0.63       482



In [327]:
scores = cross_val_score(p5, x,y,cv=10)

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [326]:
y.value_counts()

1    2629
0    2190
Name: innov_, dtype: int64

In [330]:
np.mean(scores),np.std(scores)

(0.6131999378887346, 0.030039770128867514)

In [331]:
pickle.dump?

In [333]:
with open('tfidf_lg_model.pickle', 'wb') as f:
    pickle.dump(p5, f)

In [339]:
GridSearchCV?

In [13]:
with open ('pickle/tfidf_lg_model.pickle','rb') as f:
    p5 = pickle.load(f)
p5 = GridSearchCV(p5,   param_grid= {
    'tfidf__min_df': [2,4,8],
    'tfidf__max_df': [0.75,0.8,0.85]
})

In [19]:
location = 'cachedir'
memory = Memory(location=location, verbose=10)
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p5.fit(list(x_train), y_train)
memory.clear(warn=False)
rmtree(location)

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [20]:
p5.best_params_

{'tfidf__max_df': 0.75, 'tfidf__min_df': 4}

In [21]:
p5.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='replace', dtype='int32',
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.75, max_features=None,
                                 min_df=4, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='unicode',
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lg',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ra

In [24]:
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p5.best_estimator_.fit(x_train, y_train)
print(classification_report(y_test, p5.best_estimator_.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.63      0.48      0.55       219
           1       0.64      0.77      0.70       263

    accuracy                           0.64       482
   macro avg       0.64      0.63      0.62       482
weighted avg       0.64      0.64      0.63       482



In [25]:
with open('pickle/tfidf_lg_model.pickle','wb') as f:
    pickle.dump(p5.best_estimator_, f)

In [26]:
import pickle
from sklearn.model_selection import GridSearchCV
with open ('pickle/tfidf_lg_model.pickle','rb') as f:
    p5 = pickle.load(f)
p5 = GridSearchCV(p5,   param_grid= {
    'tfidf__max_df': [0.65,0.7,0.75]
})

In [27]:
location = 'cachedir'
memory = Memory(location=location, verbose=10)
x_train ,x_test, y_train, y_test = split_data(corpus_cleaned)
p5.fit(list(x_train), y_train)
memory.clear(warn=False)
rmtree(location)

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


In [29]:
p5.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='replace', dtype='int32',
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.75, max_features=None,
                                 min_df=4, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='unicode',
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lg',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ra

In [41]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectKBest

In [35]:
tfidf = p5.best_estimator_[0]

In [49]:
p6 = Pipeline([
    ('tfidf', tfidf),
    ('select', SelectKBest(k=1000)),
    ('ada', AdaBoostClassifier(n_estimators=100))
])

In [50]:
x_train,x_test, y_train, y_test = split_data(corpus_cleaned)
p6.fit(x_train, y_train)
print(classification_report(y_test, p6.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.56      0.48      0.52       219
           1       0.61      0.69      0.65       263

    accuracy                           0.59       482
   macro avg       0.59      0.58      0.58       482
weighted avg       0.59      0.59      0.59       482



In [59]:
boosting = AdaBoostClassifier(estimators=[
    ('lr', p5.best_estimator_.named_steps.lg),
    ('bag', BaggingClassifier(DecisionTreeClassifier(criterion='entropy'), n_estimators=100,max_samples=0.5,max_features=0.1, n_jobs=-1, random_state = 42)),
    ('mnb', MultinomialNB())
], voting = 'soft')

TypeError: __init__() got an unexpected keyword argument 'estimators'

In [63]:
from sklearn.feature_extraction.text import CountVectorizer

In [64]:
CountVectorizer?

In [66]:
x_train,x_test, y_train, y_test = split_data(corpus_cleaned)
p7 = Pipeline([
    ('count', CountVectorizer(max_df=0.9,
    min_df=4,
    max_features=None,
    binary=True)), 
    ('ada', AdaBoostClassifier(base_estimator=p5.best_estimator_.named_steps.lg, n_estimators=30))
])
p7.fit(x_train, y_train)
print(classification_report(y_test, p7.predict(x_test)))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57       219
           1       0.65      0.74      0.69       263

    accuracy                           0.64       482
   macro avg       0.64      0.63      0.63       482
weighted avg       0.64      0.64      0.64       482



In [79]:
x_train,x_test, y_train, y_test = split_data(corpus_cleaned)
voting = VotingClassifier(estimators=[
    ('lr', LogisticRegression(C=1, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42, max_features=0.5)),
    ('mnb',MultinomialNB())
], voting= 'soft', weights=[1,2,1])
    
p7 = Pipeline([
    ('tfidf', tfidf), 
    ('voting', voting)
])
p7.fit(x_train, y_train)
print(classification_report(y_test, p7.predict(x_test)))

/Users/septem/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


              precision    recall  f1-score   support

           0       0.64      0.43      0.52       219
           1       0.63      0.79      0.70       263

    accuracy                           0.63       482
   macro avg       0.63      0.61      0.61       482
weighted avg       0.63      0.63      0.62       482

